In [ ]:
%pip install -r requirements.txt

In [1]:
import os
import numpy as np
import time
import constructivo as constr
import ruido as ruido
import rand_choice as aleat

In [2]:
directory_path = '../JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

In [3]:
import xlrd, xlwt
from xlutils.copy import copy as xl_copy
from xlwt import Workbook


def write_xlsx(machine_order, makespan, time, name_sheet, name_file):
    # Create a new workbook
    
    try:
        # open existing workbook
        rb = xlrd.open_workbook(name_file, formatting_info=True)
        # make a copy of it
        workbook = xl_copy(rb)
    except:
        workbook = Workbook()
    sheet = workbook.add_sheet(name_sheet)
    
    # Iterate over the matrix and write its values to the sheet
    for row_index, row in enumerate(machine_order, start=1):
        for col_index, value in enumerate(row, start=1):
            sheet.write(row_index - 1, col_index - 1, value)  # Note: row_index and col_index should start from 0
    
    # Write makespan and time to the next row
    row_index = len(machine_order)
    sheet.write(row_index, 0, int(makespan))
    sheet.write(row_index, 1, time)

    # Save the workbook to the specified file
    workbook.save(name_file)

In [4]:
#Reading all the information
for file_num in range(1, len(directory_files)+1):
  file = "JSSP" + str(file_num) + ".txt"
  with open(directory_path + "/" + file, "r" ) as f:
  #with open("../AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = max([sum([x[1] for x in machines[i]]) for i in machines])

  # Create job objects
  jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]
  jobs_ruido = [ruido.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1, 0.1) for i in range(n)]
  jobs_aleat = [aleat.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines_constr = [constr.Machine(i+1) for i in range(m)]
  machines_ruido = [ruido.Machine(i+1) for i in range(m)]
  machines_aleat = [aleat.Machine(i+1) for i in range(m)]

  # Record the start time
  constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()
  ruido_start_time = time.time(); ans_ruido, machine_order_ruido, _ = ruido.aleatorio(jobs_ruido, machines_ruido, processing_time, machines_required); ruido_end_time = time.time()
  aleat_start_time = time.time(); ans_aleat, machine_order_aleat = aleat.eleccion_aleatoria(jobs_aleat, machines_aleat); aleat_end_time = time.time()
  
  #Save the results in excel worksheets
  write_xlsx(machine_order_constr, ans_constr, constr_end_time - constr_start_time, file, "JSSP_coberndorm_Constructivo.xlsx")
  write_xlsx(machine_order_ruido, ans_ruido, ruido_end_time - ruido_start_time, file, "JSSP_coberndorm_Ruido.xlsx")
  write_xlsx(machine_order_aleat, ans_aleat, aleat_end_time - aleat_start_time, file, "JSSP_coberndorm_Aleatorio.xlsx")

  print(file, ": ", ans_constr, "    ", ans_ruido, "    ", ans_aleat, "     ", cota_inferior)

JSSP1.txt :  1462      1375      1414       977
JSSP2.txt :  1446      1407      1380       919
JSSP3.txt :  1865      1721      1659       1139
JSSP4.txt :  1667      1580      1625       1251
JSSP5.txt :  2175      2000      1973       1182
JSSP6.txt :  1965      1854      1912       1240
JSSP7.txt :  2335      2152      2115       1764
JSSP8.txt :  2432      2261      2300       1774
JSSP9.txt :  2499      2378      2541       1830
JSSP10.txt :  2710      2497      2495       1761
JSSP11.txt :  3856      3272      3264       2760
JSSP12.txt :  3266      3135      3261       2756
JSSP13.txt :  3606      3367      3561       2868
JSSP14.txt :  3639      3436      3514       2848
JSSP15.txt :  6232      5990      6132       5464
JSSP16.txt :  5973      5758      5648       5181


In [5]:
with open('cotas.txt', 'r') as f:
    cotas = f.readlines()
    cotas = [int(line.strip()) for line in cotas]   

In [6]:
import os
import xlrd

# Get a list of Excel files in the current directory
directory_files = os.listdir()
directory_files.sort()
directory_files = [file for file in directory_files if file.endswith('.xlsx')]

for file in directory_files:
    # Specify the path to your Excel file
    excel_file_path = file
    print("Current method: ", excel_file_path)
    average_distance_to_limit = []

    for i in range(1,16+1):
        sheet_name = f"JSSP{i}.txt"  # Change this to the desired sheet name
        column_letter = "B"      # Change this to the desired column letter

        # Open the Excel workbook
        workbook = xlrd.open_workbook(excel_file_path, formatting_info=True)

        # Select the specific sheet by name
        sheet = workbook.sheet_by_name(sheet_name)

        # Find the last row with data in the specified column (column B)
        max_row = sheet.nrows -1

        # Read the value from the last cell in column B
        makespan_val = sheet.cell_value(max_row, 0)
        # Read the value from the last cell in column B
        time_val = sheet.cell_value(max_row, 1)

        cota_file = cotas[i-1]

        efficacy = (makespan_val - cota_file)/cota_file
        average_distance_to_limit.append(efficacy)

        print(sheet_name)
        print(int(makespan_val))
        print(round(efficacy, 4))
        print()

    print("average distance to limit: ", round(sum(average_distance_to_limit)/len(average_distance_to_limit), 4))
    

Current method:  JSSP_coberndorm_Aleatorio.xlsx
JSSP1.txt
1414
0.4268

JSSP2.txt
1380
0.401

JSSP3.txt
1659
0.4095

JSSP4.txt
1625
0.3342

JSSP5.txt
1973
0.6132

JSSP6.txt
1912
0.5055

JSSP7.txt
2115
0.1634

JSSP8.txt
2300
0.2514

JSSP9.txt
2541
0.3545

JSSP10.txt
2495
0.3985

JSSP11.txt
3264
0.1554

JSSP12.txt
3261
0.0866

JSSP13.txt
3561
0.2208

JSSP14.txt
3514
0.1912

JSSP15.txt
6132
0.1099

JSSP16.txt
5648
-0.0049

average distance to limit:  0.2886
Current method:  JSSP_coberndorm_Constructivo.xlsx
JSSP1.txt
1462
0.4753

JSSP2.txt
1446
0.468

JSSP3.txt
1865
0.5845

JSSP4.txt
1667
0.3686

JSSP5.txt
2175
0.7784

JSSP6.txt
1965
0.5472

JSSP7.txt
2335
0.2844

JSSP8.txt
2432
0.3232

JSSP9.txt
2499
0.3321

JSSP10.txt
2710
0.5191

JSSP11.txt
3856
0.365

JSSP12.txt
3266
0.0883

JSSP13.txt
3606
0.2362

JSSP14.txt
3639
0.2336

JSSP15.txt
6232
0.128

JSSP16.txt
5973
0.0523

average distance to limit:  0.3615
Current method:  JSSP_coberndorm_Ruido.xlsx
JSSP1.txt
1375
0.3875

JSSP2.txt
1407
0.